In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"
# !pip install accelerate peft bitsandbytes transformers trl
# You only need to run this once per machine
import locale
print(locale.getdefaultlocale())

!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets matplotlib
!pip install trl
!pip install datasets

('en_US', 'UTF-8')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 12.5 MB/s eta 0:00:00
    

In [3]:
from huggingface_hub import notebook_login
notebook_login()

Load the required packages.

In [4]:
!pip uninstall torch
!pip install torch
!pip install torch-utils

Found existing installation: torch 2.1.0+cu121
Uninstalling torch-2.1.0+cu121:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.10/dist-packages/functorch/*
    /usr/local/lib/python3.10/dist-packages/nvfuser/*
    /usr/local/lib/python3.10/dist-packages/torch-2.1.0+cu121.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch/*
    /usr/local/lib/python3.10/dist-packages/torchgen/*
Proceed (Y/n)? y
  Successfully uninstalled torch-2.1.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 771.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import torch

In [6]:
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

In [7]:
#dataset="/content/training_set"
model_id="Open-Orca/Mistral-7B-OpenOrca" #"TinyLlama/TinyLlama-1.1B-Chat-v1.0"
output_model= "/content/drive/MyDrive/Bioinformatics_401_Files/BiOrca_weights" #"tinyllama"
new_model = "/content/drive/MyDrive/Bioinformatics_401_Files/BioOrca"

### Data preparation

In [8]:
# # we need to reformat the data in teh ChatML format.

# def formatted_train(input,response)->str:
#     return f"<|im_start|>user\n{input}<|im_end|>\n<|im_start|>assistant\n{response}<|im_end|>\n"

In [9]:
# def prepare_train_data(data_id):
#     data = load_dataset(data_id, split="train")
#     data_df = data.to_pandas()
#     data_df["text"] = data_df[["description", "color"]].apply(lambda x: "<|im_start|>user\n" + x["description"] + " <|im_end|>\n<|im_start|>assistant\n" + x["color"] + "<|im_end|>\n", axis=1)
#     data = Dataset.from_pandas(data_df)
#     return data

In [8]:
import json
import csv

main_prompt = """
The task is to extract biology-related triples from scientific research papers.
The rules are:
1. Only use the following predicates in the triple: “causes”, ”biolocation is”, “exposed through”, “sourced through”, “has role of”, “involved in”.
2. If there is more than one noun in the object, separate it into multiple triples.
3. If you don't find relevant biology-related triples in the paper or you are not sure, return: null.
4. The response is an array of the relevant triples in the form: [subject, predicate, object].
Q: Interaction of TMPC with ANXA2 mediated attachment and colonization of S. anginosus and induced mitogen-activated protein kinase (MAPK) activation.
A: ["TMPC", "involved in", "MAPK activation"],
["ANXA2", "involved in", "MAPK activation"]
Q: α-Lipoic acid plays an essential role in mitochondrial dehydrogenase reactions.
A: ["alpha-Lipoic acid", "involved in", "mitochondrial dehydrogenase reactions"]
Q: Ferroptosis, a form of regulated cell death that is driven by iron-dependent phospholipid peroxidation, has been implicated in multiple diseases, including cancer
A: ["Ferroptosis", "causes", "cancer"]
Q: These transporters and other biotin-binding proteins partition biotin to the cytoplasm and mitochondria cell compartments.
A: ["Biotin", "biolocation is", "cytoplasm"],
["Biotin", "biolocation is", "mitochondria"]
"""


def jsonl_to_csv(jsonl_file, csv_file):
    with open(jsonl_file, 'r', encoding='utf-8') as f:
        data = f.readlines()

    with open(csv_file, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["text"])

        for line in data:
            record = json.loads(line.strip())
            human_input = record['input']
            response = '\n'.join([repr(inner_lst) for inner_lst in record['output']])
            placeholder_prompt = f"""
            Q: {human_input}
            A:
            """
            writer.writerow([f"### Human: {main_prompt} {placeholder_prompt} ### Assistant: {response}"])

# Example usage:
jsonl_file = '/content/drive/MyDrive/Bioinformatics_401_Files/Data/train_10.jsonl'
csv_file = 'output.csv'
jsonl_to_csv(jsonl_file, csv_file)


In [9]:
#data = prepare_train_data(dataset)
import pandas as pd
# df=pd.read_csv("TMEM106B_data_example_3_96.csv")
df=pd.read_csv("output.csv")
data = Dataset.from_pandas(df)


In [10]:
data

Dataset({
    features: ['text'],
    num_rows: 10
})

In [11]:
data[0]

{'text': '### Human: \nThe task is to extract biology-related triples from scientific research papers.\nThe rules are:\n1. Only use the following predicates in the triple: “causes”, ”biolocation is”, “exposed through”, “sourced through”, “has role of”, “involved in”.\n2. If there is more than one noun in the object, separate it into multiple triples.\n3. If you don\'t find relevant biology-related triples in the paper or you are not sure, return: null.\n4. The response is an array of the relevant triples in the form: [subject, predicate, object]. \nQ: Interaction of TMPC with ANXA2 mediated attachment and colonization of S. anginosus and induced mitogen-activated protein kinase (MAPK) activation.\nA: ["TMPC", "involved in", "MAPK activation"],\n["ANXA2", "involved in", "MAPK activation"]\nQ: α-Lipoic acid plays an essential role in mitochondrial dehydrogenase reactions.\nA: ["alpha-Lipoic acid", "involved in", "mitochondrial dehydrogenase reactions"]\nQ: Ferroptosis, a form of regulate

### Model the Model (not the base version)

In [72]:
def get_model_and_tokenizer(mode_id):

    tokenizer = AutoTokenizer.from_pretrained(mode_id)
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto"
    )
    model.config.use_cache=False
    model.config.pretraining_tp=1
    return model, tokenizer

In [15]:
#!pip install -i https://test.pypi.org/simple/bitsandbytes

In [17]:
model, tokenizer = get_model_and_tokenizer(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

...

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

### Setting up the LoRA

In [73]:
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )

In [84]:
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=4, #16,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=3,
        max_steps=250,
        fp16=False,
        # push_to_hub=True
    )

In [85]:
trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=5000,
    )

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [63]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            12Gi       4.4Gi       254Mi        20Mi       8.0Gi       8.0Gi
Swap:             0B          0B          0B


In [86]:
trainer.train()

Step,Training Loss
10,1.807800
20,0.475400
30,0.219900
40,0.124600
50,0.104000
60,0.094700
70,0.083100
80,0.069700
90,0.056900
100,0.047200


KeyboardInterrupt: 

In [37]:
trainer.model.save_pretrained(new_model)

In [ ]:
!pip install safetensors
import safetensors

### Merging the LoRA with the base model

In [ ]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM
import torch
import os

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, load_in_8bit=False,
                                             device_map="auto",
                                             trust_remote_code=True)

model_path = "BiOrca/checkpoint-250" #"/content/tinyllama/checkpoint-250"

peft_model = PeftModel.from_pretrained(model, model_path, from_transformers=True, device_map="auto")

model = peft_model.merge_and_unload()

In [ ]:
model

### Inference from the LLM

In [ ]:
from transformers import GenerationConfig
from time import perf_counter

def generate_response(user_input):

  prompt = formatted_prompt(user_input)

  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.5,do_sample = True,
      top_k=1,temperature=0.1,repetition_penalty=1.2,
      max_new_tokens=120,pad_token_id=tokenizer.eos_token_id
  )
  start_time = perf_counter()

  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

  outputs = model.generate(**inputs, generation_config=generation_config)
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  output_time = perf_counter() - start_time
  print(f"Time taken for inference: {round(output_time,2)} seconds")

In [ ]:
# def formatted_prompt(question)-> str:
#     return f"<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant:"


def formatted_prompt(question)-> str:
    return f"### Human: {question} ### Assistant:"




In [ ]:
def print_color_space(hex_color):
    def hex_to_rgb(hex_color):
        hex_color = hex_color.lstrip('#')
        return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    r, g, b = hex_to_rgb(hex_color)
    print(f'{hex_color}: \033[48;2;{r};{g};{b}m           \033[0m')

In [ ]:
generate_response(user_input='Is TMEM106B amyloidogenic protein?')